In [31]:
import pandas as pd
import numpy as np
df=pd.read_csv("btceur_1min.csv",)
df=df[::-1]
df.reset_index(inplace=True,drop=True)

C:\Users\divya\AppData\Local\Temp\ipykernel_28312\3401914393.py:3: DtypeWarning: Columns (1,2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("btceur_1min.csv",)


In [32]:
df.head()

,date,open,high,low,close,volume,volumeNotional,tradesDone
0,2020-02-27T00:00:00+00:00,8070.150138,8082.188471,8068.752691,8078.25844,1.979712,15992.625415,25.0
1,2020-02-27T00:01:00+00:00,8087.172509,8087.24659,8079.512835,8080.304205,3.045879,24611.627195,24.0
2,2020-02-27T00:02:00+00:00,8084.285475,8093.348403,8084.266341,8090.739578,11.306356,91476.780759,18.0
3,2020-02-27T00:03:00+00:00,8068.651426,8068.651426,8060.968785,8060.968785,4.359564,35142.306578,22.0
4,2020-02-27T00:04:00+00:00,8068.632019,8068.632019,8051.159339,8051.610936,14.036788,113018.759629,56.0


In [33]:
df = df.dropna()

In [34]:
df = df[df['date'] != 'date']

In [35]:
df=df[['date','close']]


In [36]:
df['close']=df['close'].astype(float)

In [37]:
df = df.drop_duplicates(keep='first')


In [73]:
import pandas as pd
# df=pd.read_csv("eur_1min_impute_final.csv")

# df=pd.read_csv("eru_noDUP.csv")
# Convert 'date' column to datetime
df['date'] = pd.to_datetime(df['date'],format='mixed')

# Create a complete range of 1-min intervals between min and max date
full_range = pd.date_range(start=df['date'].min(), end=df['date'].max(), freq='1T')
 
# Find missing minutes
missing_minutes = full_range.difference(df['date'])

# Display missing minutes
print(missing_minutes)

DatetimeIndex([], dtype='datetime64[ns, UTC]', freq='min')


C:\Users\divya\AppData\Local\Temp\ipykernel_28312\1484005424.py:9: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_range = pd.date_range(start=df['date'].min(), end=df['date'].max(), freq='1T')


In [74]:
diff=(df['date'][1:-2]-df['date'].shift(1)[1:-2])
print(diff)
diff=pd.Series([x.components.minutes+x.components.hours*60+x.components.days*60*24 for x in diff])

diff.unique()
# diff.to_csv("difference_eur.csv")

1         0 days 00:01:00
2         0 days 00:01:00
3         0 days 00:01:00
4         0 days 00:01:00
5         0 days 00:01:00
                ...      
2880789   0 days 00:01:00
2880790   0 days 00:01:00
2880791   0 days 00:01:00
2880792   0 days 00:01:00
2880793   0 days 00:01:00
Name: date, Length: 2881198, dtype: timedelta64[ns]


array([1], dtype=int64)

In [64]:
df.reset_index(inplace=True,drop=True)
print(diff[diff<0].index)



Index([], dtype='int64')


In [65]:
df[2069276:2069282]

,date,close
2069276,2024-02-02 23:56:00+00:00,39984.480115
2069277,2024-02-02 23:57:00+00:00,39985.708487
2069278,2024-02-02 23:58:00+00:00,40006.701130
2069279,2024-02-02 23:59:00+00:00,40012.062148
2069280,2024-02-03 00:00:00+00:00,40012.611816
2069281,2024-02-03 00:01:00+00:00,39997.841168


In [66]:
diff=pd.Series(diff)
diff[diff<0]

Series([], dtype: int64)

In [67]:
delta:pd.Timedelta=df["date"].iloc[0]-df["date"].iloc[1]
delta.components

Components(days=-1, hours=23, minutes=59, seconds=0, milliseconds=0, microseconds=0, nanoseconds=0)

In [68]:
import numpy as np
def add_missing(df,interval=1):
    imputed=0
    first=True
    data_to_impute=[]
    for i in range(len(df)-5):
        # print(i)
        if i%10000==0:
            # print(i)
            pass
        try:
            # i+=imputed
            delta:pd.Timedelta=df["date"].iloc[i+1]-df["date"].iloc[i]
            n_impute=(delta.components.days*60*24+delta.components.hours*60+delta.components.minutes-interval)//interval
        except:
            print(i,n_impute,delta.components.minutes)
        
        try:
        
            price_delta=df["close"].iloc[i+1]-df["close"].iloc[i]
        except:
            print(i,df.iloc[i-4:i+4])
        if n_impute > 0:
            if first:
                first=False
                print(i)
            impute_data=[[],[],i]
            add_delta=price_delta/(n_impute+1)
            for j in range(n_impute):
                # price_impute=(j+1+np.random.randint(0,40)/100-0.2)*add_delta 
                # date_impute=df["date"].iloc[i]+pd.Timedelta(minutes=j+1)
                impute_data[0].append(df["date"].iloc[i]+pd.Timedelta(minutes=j+1))
                impute_data[1].append((j+1+np.random.randint(0,40)/100-0.2)*add_delta+df["close"].iloc[i])
            # before=df[:i+1]
            # after=df[i+1:]
            # middle=pd.DataFrame({'date':impute_data[0],'close':impute_data[1]})
            # df=pd.concat([before,middle,after])
            data_to_impute.append([i,n_impute,impute_data[0],impute_data[1]])
            imputed+=n_impute
    print("impputed thi smany"+str(imputed))
    return df,data_to_impute

print(len(df))
print(df.head())
df.reset_index(drop=True,inplace=True)
df,data_to_impute=add_missing(df)
print(data_to_impute)

2880796
                       date        close
0 2020-02-27 00:00:00+00:00  8078.258440
1 2020-02-27 00:01:00+00:00  8080.304205
2 2020-02-27 00:02:00+00:00  8090.739578
3 2020-02-27 00:03:00+00:00  8060.968785
4 2020-02-27 00:04:00+00:00  8051.610936
2684668
impputed thi smany405
[[2684668, 405, [Timestamp('2025-04-05 08:29:00+0000', tz='UTC'), Timestamp('2025-04-05 08:30:00+0000', tz='UTC'), Timestamp('2025-04-05 08:31:00+0000', tz='UTC'), Timestamp('2025-04-05 08:32:00+0000', tz='UTC'), Timestamp('2025-04-05 08:33:00+0000', tz='UTC'), Timestamp('2025-04-05 08:34:00+0000', tz='UTC'), Timestamp('2025-04-05 08:35:00+0000', tz='UTC'), Timestamp('2025-04-05 08:36:00+0000', tz='UTC'), Timestamp('2025-04-05 08:37:00+0000', tz='UTC'), Timestamp('2025-04-05 08:38:00+0000', tz='UTC'), Timestamp('2025-04-05 08:39:00+0000', tz='UTC'), Timestamp('2025-04-05 08:40:00+0000', tz='UTC'), Timestamp('2025-04-05 08:41:00+0000', tz='UTC'), Timestamp('2025-04-05 08:42:00+0000', tz='UTC'), Timestamp('20

In [52]:
def impute(df,data,file):
    prev=0
    
    for i in data:
        if prev==0:
            df[0:i[0]+1].to_csv(file,mode="a",index=False)
        else:
            df[prev:i[0]+1].to_csv(file,mode="a",header=False,index=False)
        
        pd.DataFrame({'date':i[2],'close':i[3]}).to_csv(file,mode="a",header=False,index=False)
        prev=i[0]+1

    index=i[0]+i[1]+1
    df[index:].to_csv(file,mode="a",header=False,index=False)
    
impute(df,data_to_impute,"eur_1min_impute.csv")

In [70]:
def impute_in_df(df:pd.DataFrame,data):
    prev=0
    new_df=df.copy(deep=True)
    
    for i in data:
        # if prev==0:
        #     prev=df[0:i[0]+1]
        #     # df[0:i[0]+1].to_csv(file,mode="a",index=False)
        # else:
        #     df[prev:i[0]+1].to_csv(file,mode="a",header=False,index=False)
        p=df[0:i[0]+1]
        mid=pd.DataFrame({'date':i[2],'close':i[3]})
        mid['date'] = pd.to_datetime(mid['date'],format='mixed')
        new_df=pd.concat([new_df[:i[0]+1],mid,new_df[i[0]+1:]])
        # pd.DataFrame({'date':i[2],'close':i[3]}).to_csv(file,mode="a",header=False,index=False)
    return new_df
   
ini=len(df)
print(ini)
df=impute_in_df(df,data_to_impute)
print(len(df)-ini)


2880796
405


In [75]:
df.to_csv("eur_1min_impute_final.csv",index=False)

In [70]:
a=[1,2,3,4,5]
print(a[:2],a[2:])

[1, 2] [3, 4, 5]


In [73]:
a=[[1,2,3,4],[5,6,77,9]]
a=pd.DataFrame({'a':a[0],'b':a[1]})[::-1].reset_index(drop=True)
a[3:]

,a,b
3,1,5


In [ ]:
df.loc[:df['date'][5]]

TypeError: cannot do slice indexing on RangeIndex with these indexers [2020-02-29 00:05:00+00:00] of type Timestamp

In [72]:
for i in range(5):
    pass

print(i)

4


In [67]:
df=pd.read_csv("eru_noDUP.csv")


In [68]:
df

,Unnamed: 0,date,close
0,0,2020-02-27 00:00:00+00:00,8078.258440
1,1,2020-02-27 00:01:00+00:00,8080.304205
2,2,2020-02-27 00:02:00+00:00,8090.739578
3,3,2020-02-27 00:03:00+00:00,8060.968785
4,4,2020-02-27 00:04:00+00:00,8051.610936
...,...,...,...
2869035,2869035,2025-08-19 19:56:00+00:00,97210.049853
2869036,2869036,2025-08-19 19:57:00+00:00,97203.305855
2869037,2869037,2025-08-19 19:58:00+00:00,97159.075204
2869038,2869038,2025-08-19 19:59:00+00:00,97118.749091


In [70]:
df.drop(['Unnamed: 0'],inplace=True,axis=1)
df.reset_index(inplace=True,drop=True)

In [72]:
df.to_csv('eru_noDUP.csv',index=False)